In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
df_Finland= pd.read_csv("df_Finland.csv" ,encoding='unicode_escape')
df_Finland

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Finland,2020/1/1,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Finland,2020/1/2,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Finland,2020/1/3,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Finland,2020/1/4,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Finland,2020/1/5,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Finland,2021/9/24,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007685
728,Finland,2021/9/25,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007692
729,Finland,2021/9/26,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007668
730,Finland,2021/9/27,5540720,86%,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007653


In [3]:
#shift 21days mortality
df_Finland_mortality=df_Finland["mortality"]
df_Finland_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
         ...   
727    0.007685
728    0.007692
729    0.007668
730    0.007653
731    0.000000
Name: mortality, Length: 710, dtype: float64

In [5]:
#删除后21天
df_Finland.drop(df_Finland.index[710:732], inplace=True)
df_Finland

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Finland,2020/1/1,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Finland,2020/1/2,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Finland,2020/1/3,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Finland,2020/1/4,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Finland,2020/1/5,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,Finland,2021/9/2,5540720,86%,28552.0,6694382.0,5.146,1206.551,24557.0,4.426,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.008021
706,Finland,2021/9/3,5540720,86%,25395.0,6719777.0,4.577,1211.128,24023.0,4.330,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007986
707,Finland,2021/9/4,5540720,86%,20015.0,6739792.0,3.607,1214.736,25067.0,4.518,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007980
708,Finland,2021/9/5,5540720,86%,14411.0,6754203.0,2.597,1217.333,24592.0,4.432,...,0.0,2.0,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007986


In [7]:
df_Finland1=df_Finland.join(df_Finland_mortality,how="left",rsuffix="after21days")
df_Finland1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days,mortalityafter21days
0,Finland,2020/1/1,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,Finland,2020/1/2,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,Finland,2020/1/3,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,Finland,2020/1/4,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,Finland,2020/1/5,5540720,86%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,Finland,2021/9/2,5540720,86%,28552.0,6694382.0,5.146,1206.551,24557.0,4.426,...,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.008021,0.008021,0.008021
706,Finland,2021/9/3,5540720,86%,25395.0,6719777.0,4.577,1211.128,24023.0,4.330,...,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007986,0.007986,0.007986
707,Finland,2021/9/4,5540720,86%,20015.0,6739792.0,3.607,1214.736,25067.0,4.518,...,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007980,0.007980,0.007980
708,Finland,2021/9/5,5540720,86%,14411.0,6754203.0,2.597,1217.333,24592.0,4.432,...,2.0,2.0,0.0,0.0,1.0,5.0,2.0,0.007986,0.007986,0.007986


In [8]:
df_Finland.to_csv("df_Finland_after21days",index=False)